In [4]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 70.2 MB/s eta 0:00:00


In [5]:
import pdfplumber
import io
from tabulate import tabulate
from google.colab import auth
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload

Na célula seguinte, você deve colocar o id da pasta do Google Drive onde estão os arquivos PDFs. Note que o id é o valor que aparece na URL: https://drive.google.com/drive/folders/ID

In [6]:
drive_id = "13-g2FSSOlv2gGWRceoRwO7nJGKSUoICQ"

Na célula seguinte, você deve colocar todos os prefixos ou palavras que deseja buscar nos arquivos PDFs. Exemplo: filtro=\["desuman", "human", "animal", "animais", "clima"\]

In [7]:
filtro = ["desuman", "human", "animal", "animais", "bestial", "monstro", "clima", "mudanças climáticas", "crise climática", "racis", "branco", "branquitude"]

In [9]:
escopo = [
        "https://www.googleapis.com/auth/drive.appdata",
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"
    ]
auth.authenticate_user()
creds, _ = google.auth.default(scopes=escopo)

servico_drive = build("drive", "v3", credentials=creds)

In [14]:
def analisar_arquivo(pdf: pdfplumber.PDF):
  contador_ocorrencias = dict.fromkeys(filtro, 0)
  numerador_paginas = dict.fromkeys(filtro, '')

  for pagina in pdf.pages:
    for prefixo in filtro:
      ocorrencias = pagina.search(prefixo, False, False)
      if len(ocorrencias) > 0:
        contador_ocorrencias[prefixo] += len(ocorrencias)
        numerador_paginas[prefixo] += str(pagina.page_number) + ','

  tabela = []
  for prefixo in filtro:
    tabela.append([prefixo, contador_ocorrencias[prefixo], numerador_paginas[prefixo][:-1]])
  return tabulate(tabela, headers=["Excerto", "Ocorrências", "Páginas"], tablefmt="pipe")

In [10]:
# Retorna os arquivos ou pastas encontradas na pasta de ID id_pasta
def pesquisar_arquivos_pasta(id_pasta: str):
  arquivos = []

  query = f"'{id_pasta}' in parents"

  resposta_api = (
      servico_drive.files().list(
          q=query,
          fields="files(id, name, mimeType)",
          supportsAllDrives=True,
          includeItemsFromAllDrives=True
      ).execute()
  )

  for item in resposta_api.get("files", []):
    if item['mimeType'] == 'application/vnd.google-apps.folder':
      arquivos.extend(pesquisar_arquivos_pasta(item['id']))
    else:
      arquivos.append(item)

  return arquivos

In [11]:
arquivos = pesquisar_arquivos_pasta(drive_id)
if not arquivos:
  print("Nenhum arquivo encontrado na pasta do Drive.")
else:
  print(f"{len(arquivos)} arquivos encontrados.")

43 arquivos encontrados.


In [15]:
for arquivo in arquivos:
  if arquivo['mimeType'] == 'application/pdf':
    print(f"\tTentando baixar o {arquivo['name']}")

    # Baixar arquivo do Drive como I/O
    request = servico_drive.files().get_media(fileId=arquivo['id'])
    file = io.BytesIO()
    downloader = MediaIoBaseDownload(file, request)
    done = False
    while done is False:
      status, done = downloader.next_chunk()

      # Checar se o arquivo é um PDF válido e fazer o fichamento
      try:
        with pdfplumber.open(file) as pdf:
          print(f"\t\tArquivo válido como PDF, iniciando análise...")
          tabela = analisar_arquivo(pdf)

          # Abrir o arquivo markdown para salvar o título e tabela
          try:
            with open("/content/output.md", "a", encoding="utf-8") as output:
              titulo = ""
              # Essa regra confere se o nome do arquivo está na formatação "OK Autoria - Título.pdf"
              if arquivo['name'].startswith("OK "):
                titulo = arquivo['name'][3:-4]
              else:
                titulo = arquivo['name'][:-4]
              output.write(f"\n# {titulo} \n")
              output.write(tabela)
              print("\t\tSalvou com sucesso a análise desse PDF no arquivo markdown!")
          except Exception as error:
            print(f"\t\tOcorreu um erro na hora de salvar a análise, tente novamente")
            print(error)
      except Exception as error:
        print(f"\t\tPDF inválido: {arquivo['name']}")
        print(error)

	Tentando baixar o OK SIMIÃO - Crise climática, mecanismos de mercado e a financeirização da natureza.pdf
		Arquivo válido como PDF, iniciando análise...
		Salvou com sucesso a análise desse PDF no arquivo markdown!
	Tentando baixar o OK ZACARIAS - A lógica destrutiva do capital crise ambiental e mudanças climáticas.pdf
		Arquivo válido como PDF, iniciando análise...
		Salvou com sucesso a análise desse PDF no arquivo markdown!
	Tentando baixar o OK SILVA - Corpos antropofágicos - Supermáquina e interseccionalidades em cartoescrita de fluxos indisciplinares.pdf
		Arquivo válido como PDF, iniciando análise...
		Salvou com sucesso a análise desse PDF no arquivo markdown!
	Tentando baixar o OK PEREIRA - Estudos Multiespécies.pdf
		Arquivo válido como PDF, iniciando análise...
		Salvou com sucesso a análise desse PDF no arquivo markdown!
	Tentando baixar o OK REIS - O especismo na mídia.pdf
		Arquivo válido como PDF, iniciando análise...
		Salvou com sucesso a análise desse PDF no arquivo 

No próximo passo, o arquivo **output.md** que está localmente salvo na seção "Arquivos" (ver ícone de pasta no menu lateral esquerdo), será importado já em formato Google Docs na pasta informada anteriormente.

In [16]:
# Salvar o arquivo markdown das análises no Google Drive
metadados_markdown = {
    "name": "Análises de prefixo - Ferramenta de extração de PDF",
    "mimeType": "application/vnd.google-apps.documents",
    "parents": [
        f"{drive_id}"
        ]
    }
midia_markdown = MediaFileUpload("/content/output.md", mimetype="text/markdown")
upload_arquivo_fichamento = servico_drive.files().create(
    body=metadados_markdown,
    media_body=midia_markdown,
    supportsAllDrives=True,
    fields="id"
    ).execute()

print(f'Arquivo salvo no Google Drive: {upload_arquivo_fichamento.get("id")} ! Boa análise!')


Arquivo salvo no Google Drive: 12R0UpZBBRTFJs8f-OjCAv4du3xaHCeGnl_hDywvtjgk ! Boa análise!
